In [4]:
import pandas as pd
import os
import numpy as np

In [11]:
url_davos = "https://data.geo.admin.ch/ch.meteoschweiz.ogd-nbcn/dav/ogd-nbcn_dav_m.csv"
url_sion = "https://data.geo.admin.ch/ch.meteoschweiz.ogd-nbcn/sio/ogd-nbcn_sio_m.csv"
url_metadata = "https://data.geo.admin.ch/ch.meteoschweiz.ogd-nbcn/ogd-nbcn_meta_parameters.csv"
folder_path = "project-glaciers/data"


In [12]:
try:
    metadata = pd.read_csv(url_metadata, delimiter=';', encoding='latin1')
except UnicodeDecodeError:
    try:
        metadata = pd.read_csv(url_metadata, delimiter=';', encoding='cp1252')
    except UnicodeDecodeError:
        try:
            metadata = pd.read_csv(url_metadata, delimiter=';', encoding='utf-16')
        except Exception as e:
            print(f"Failed to read file: {e}")

metadata = metadata[['parameter_shortname',
                         'parameter_description_en',
                         'parameter_unit'
                    ]]

In [13]:
sion_weather = pd.read_csv(url_sion, delimiter = ';')
sion_weather['date'] = pd.to_datetime(
    sion_weather['reference_timestamp'],
    format='%d.%m.%Y %H:%M'
)

sion_weather = sion_weather.drop(columns = ['station_abbr'])
sion_weather = sion_weather.set_index('date')

davos_weather = pd.read_csv(url_davos, delimiter = ';')
davos_weather['date'] = pd.to_datetime(
    davos_weather['reference_timestamp'],
    format='%d.%m.%Y %H:%M'
)
davos_weather = davos_weather.drop(columns = ['station_abbr'])
davos_weather = davos_weather.set_index('date')

sion_1914 = sion_weather[
    (sion_weather.index >= '1914-10-01') &
    (sion_weather.index < '2025-10-01')
]
sion_1914 = sion_1914.dropna(axis=1)
sion_1914 = sion_1914.drop('reference_timestamp', axis=1)


davos_1914 = davos_weather[
    (davos_weather.index >= '1914-10-01') &
    (davos_weather.index < '2025-10-01')
]
davos_1914 = davos_1914.dropna(axis=1)
davos_1914 = davos_1914.drop('reference_timestamp', axis=1)

In [14]:
metadata = metadata[metadata['parameter_shortname'].isin(davos_1914.columns)]
metadata = metadata.reset_index(drop = True)
metadata.to_csv(os.path.join(folder_path, "weather_metadata.csv"), index = False)
sion_1914.to_csv(os.path.join(folder_path, "weather_data_sion_monthly.csv"), index = True)
davos_1914.to_csv(os.path.join(folder_path, "weather_data_davos_monthly.csv"), index = True)

In [15]:
davos = davos_1914[['rhs150m0', 'shs000m0', 'ths200m0', 'ths2dymx', 'ths2dymn']].copy()
davos.rename(columns = {'rhs150m0': 'precipitation (mm)'}, inplace = True)
davos.rename(columns = {'shs000m0': 'sunshine (min)'}, inplace = True)
davos.rename(columns = {'ths200m0': 'temperature (°C)'}, inplace = True)
davos.rename(columns = {'ths2dymx': 'max_temperature (°C)'}, inplace = True)
davos.rename(columns = {'ths2dymn': 'min_temperature (°C)'}, inplace = True)

sion = sion_1914[['rhs150m0', 'shs000m0', 'ths200m0']].copy()
sion.rename(columns = {'rhs150m0': 'precipitation (mm)'}, inplace = True)
sion.rename(columns = {'shs000m0': 'sunshine (min)'}, inplace = True)
sion.rename(columns = {'ths200m0': 'temperature (°C)'}, inplace = True)

In [16]:
sion_p_s = sion[['precipitation (mm)', 'sunshine (min)']].copy()
hy_data_sion_p_s = sion_p_s.groupby(np.arange(len(sion_p_s)) // 12).sum()

# Generate the date range for hydrological years (October to September)
start_date = '1914-10-01'
end_date = '2024-10-01'
hy_dates = pd.date_range(start = start_date, end = end_date, freq = 'YS-OCT')  # 'AS-OCT' = Annual, starting in October

# Assign the date range as the index
hy_data_sion_p_s.index = hy_dates


sion_temp = sion[['temperature (°C)']].copy()
sion_temp["days_in_month"] = sion_temp.index.days_in_month
sion_temp['temp*days'] = sion_temp['temperature (°C)'] * sion_temp['days_in_month']

n = 12  # Number of rows per group (12 months)
sion_temp = pd.DataFrame({
    'mean temperature (°C)': [
        sion_temp['temp*days'].iloc[i:i+n].sum() / sion_temp['days_in_month'].iloc[i:i+n].sum()
        for i in range(0, len(sion_temp), n)
    ]
})

sion_temp = sion_temp.round(1)
sion_temp.index = hy_dates


hy_data_sion = pd.concat([hy_data_sion_p_s, sion_temp], axis=1)
hy_data_sion = hy_data_sion.round(1)
hy_data_sion["date"] = hy_data_sion.index

In [17]:
# Get sums of precipitation and sunchine for the hydrological years
davos_p_s = davos[['precipitation (mm)', 'sunshine (min)']].copy()
hy_data_davos_p_s = davos_p_s.groupby(np.arange(len(davos_p_s)) // 12).sum()

# Generate the date range for hydrological years (October to September)
start_date = '1914-10-01'
end_date = '2024-10-01'
hy_dates = pd.date_range(start = start_date, end = end_date, freq = 'YS-OCT')  # 'AS-OCT' = Annual, starting in October

# Assign the date range as the index
hy_data_davos_p_s.index = hy_dates


# Compute average temperature for the hydrological years
davos_temp = davos[['temperature (°C)']].copy()
davos_temp["days_in_month"] = davos_temp.index.days_in_month
davos_temp['temp*days'] = davos_temp['temperature (°C)'] * davos_temp['days_in_month']

n = 12  # Number of rows per group (12 months)
davos_temp = pd.DataFrame({
    'mean temperature (°C)': [
        davos_temp['temp*days'].iloc[i:i+n].sum() / davos_temp['days_in_month'].iloc[i:i+n].sum()
        for i in range(0, len(davos_temp), n)
    ]
})

davos_temp = davos_temp.round(1)
davos_temp.index = hy_dates


# Compute average daily max temperature for the hydrological years
davos_temp_max = davos[['max_temperature (°C)']].copy()
davos_temp_max["days_in_month"] = davos_temp_max.index.days_in_month
davos_temp_max['temp*days'] = davos_temp_max['max_temperature (°C)'] * davos_temp_max['days_in_month']

n = 12  # Number of rows per group (12 months)
davos_temp_max = pd.DataFrame({
    'mean max daily temperature (°C)': [
        davos_temp_max['temp*days'].iloc[i:i+n].sum() / davos_temp_max['days_in_month'].iloc[i:i+n].sum()
        for i in range(0, len(davos_temp_max), n)
    ]
})

davos_temp_max = davos_temp_max.round(1)
davos_temp_max.index = hy_dates


# Compute average daily min temperature for the hydrological years
davos_temp_min = davos[['min_temperature (°C)']].copy()
davos_temp_min["days_in_month"] = davos_temp_min.index.days_in_month
davos_temp_min['temp*days'] = davos_temp_min['min_temperature (°C)'] * davos_temp_min['days_in_month']

n = 12  # Number of rows per group (12 months)
davos_temp_min = pd.DataFrame({
    'mean min daily temperature (°C)': [
        davos_temp_min['temp*days'].iloc[i:i+n].sum() / davos_temp_min['days_in_month'].iloc[i:i+n].sum()
        for i in range(0, len(davos_temp_min), n)
    ]
})

davos_temp_min = davos_temp_min.round(1)
davos_temp_min.index = hy_dates


hy_data_davos = pd.concat([hy_data_davos_p_s, davos_temp, davos_temp_max, davos_temp_min], axis=1)
hy_data_davos = hy_data_davos.round(1)
hy_data_davos['date'] = hy_data_davos.index
hy_data_davos = hy_data_davos.reset_index(drop = True)


In [18]:
hy_data_sion.to_csv(os.path.join(folder_path, "weather_data_sion_hy.csv"), index = False)
hy_data_davos.to_csv(os.path.join(folder_path, "weather_data_davos_hy.csv"), index = False)

In [19]:
# Define winter and summer months
winter_months = [10, 11, 12, 1, 2, 3, 4]   # Oct–Apr
summer_months = [5, 6, 7, 8, 9]            # May–Sep

def classify_season(date):
    if date.month in winter_months:
        return 'winter'
    else:
        return 'summer'

def season_year(date):
    # Winter belongs to the year of January (e.g. winter 1914-15 -> 1915)
    if date.month >= 10:
        return date.year + 1  # Oct–Dec -> next year
    else:
        return date.year

sion_seasonal = sion.copy()
sion_seasonal['season'] = sion_seasonal.index.to_series().apply(classify_season)
sion_seasonal['season_year'] = sion_seasonal.index.to_series().apply(season_year)

seasonal_sum = (
    sion_seasonal[['precipitation (mm)', 'sunshine (min)', 'season', 'season_year']]
    .groupby(['season_year', 'season'])
    .sum()
)

sion_seasonal['days_in_month'] = sion_seasonal.index.days_in_month
sion_seasonal['temp*days'] = sion_seasonal['temperature (°C)'] * sion_seasonal['days_in_month']

seasonal_temp = (
    sion_seasonal.groupby(['season_year', 'season'])
    .apply(lambda df: df['temp*days'].sum() / df['days_in_month'].sum())
    .to_frame(name='mean temperature (°C)')
)

seasonal_data = pd.concat([seasonal_sum, seasonal_temp], axis=1)
seasonal_data = seasonal_data.round(1)

def compute_season_date(row):
    season_year, season = row.name  # MultiIndex: (season_year, season)
    
    if season == "winter":
        return pd.Timestamp(season_year - 1, 10, 1)
    else:  # summer
        return pd.Timestamp(season_year, 5, 1)

seasonal_data['date'] = seasonal_data.apply(compute_season_date, axis=1)
# Make "date" the new index
seasonal_data = seasonal_data.set_index('date')

# Sort chronologically (important!)
seasonal_data = seasonal_data.sort_index()
seasonal_data = seasonal_data.reset_index()

# Even rows → winter (0, 2, 4, ...)
weather_sion_winter = seasonal_data.iloc[::2, :]  

# Odd rows → summer (1, 3, 5, ...)
weather_sion_summer = seasonal_data.iloc[1::2, :]

weather_sion_summer = weather_sion_summer.reset_index(drop=True)
weather_sion_winter = weather_sion_winter.reset_index(drop=True)
weather_sion_summer

/tmp/ipykernel_1087/2165840859.py:33: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: df['temp*days'].sum() / df['days_in_month'].sum())


,date,precipitation (mm),sunshine (min),mean temperature (°C)
0,1915-05-01,213.3,70774.0,15.7
1,1916-05-01,210.4,67496.0,14.8
2,1917-05-01,246.4,74086.0,16.7
3,1918-05-01,271.1,77064.0,15.8
4,1919-05-01,124.4,83162.0,16.4
...,...,...,...,...
106,2021-05-01,376.1,70743.0,17.6
107,2022-05-01,194.6,83436.0,20.2
108,2023-05-01,231.8,77947.0,19.9
109,2024-05-01,279.3,67714.0,18.5


In [20]:
davos_seasonal = davos.copy()
davos_seasonal['season'] = davos_seasonal.index.to_series().apply(classify_season)
davos_seasonal['season_year'] = davos_seasonal.index.to_series().apply(season_year)

seasonal_sum = (
    davos_seasonal[['precipitation (mm)', 'sunshine (min)', 'season', 'season_year']]
    .groupby(['season_year', 'season'])
    .sum()
)

davos_seasonal['days_in_month'] = davos_seasonal.index.days_in_month
davos_seasonal['temp*days'] = davos_seasonal['temperature (°C)'] * sion_seasonal['days_in_month']
davos_seasonal['temp_max*days'] = davos_seasonal['max_temperature (°C)'] * sion_seasonal['days_in_month']
davos_seasonal['temp_min*days'] = davos_seasonal['min_temperature (°C)'] * sion_seasonal['days_in_month']


seasonal_temp_d= (
    davos_seasonal.groupby(['season_year', 'season'])
    .apply(lambda df: df['temp*days'].sum() / df['days_in_month'].sum())
    .to_frame(name='mean temperature (°C)')
)


seasonal_temp_d_max= (
    davos_seasonal.groupby(['season_year', 'season'])
    .apply(lambda df: df['temp_max*days'].sum() / df['days_in_month'].sum())
    .to_frame(name='mean max daily temperature (°C)')
)

seasonal_temp_d_min= (
    davos_seasonal.groupby(['season_year', 'season'])
    .apply(lambda df: df['temp_min*days'].sum() / df['days_in_month'].sum())
    .to_frame(name='mean min daily temperature (°C)')
)


seasonal_data_d = pd.concat([seasonal_sum, seasonal_temp_d, seasonal_temp_d_max, seasonal_temp_d_min], axis=1)
seasonal_data_d = seasonal_data_d.round(1)

def compute_season_date(row):
    season_year, season = row.name  # MultiIndex: (season_year, season)
    
    if season == "winter":
        return pd.Timestamp(season_year - 1, 10, 1)
    else:  # summer
        return pd.Timestamp(season_year, 5, 1)

seasonal_data_d['date'] = seasonal_data_d.apply(compute_season_date, axis=1)
# Make "date" the new index
seasonal_data_d = seasonal_data_d.set_index('date')

# Sort chronologically (important!)
seasonal_data_d = seasonal_data_d.sort_index()
seasonal_data_d = seasonal_data_d.reset_index()


# Even rows → winter (0, 2, 4, ...)
weather_davos_winter = seasonal_data_d.iloc[::2, :]  

# Odd rows → summer (1, 3, 5, ...)
weather_davos_summer = seasonal_data_d.iloc[1::2, :]

weather_davos_summer = weather_davos_summer.reset_index(drop=True)
weather_davos_winter = weather_davos_winter.reset_index(drop=True)
weather_davos_summer

/tmp/ipykernel_1087/3573682381.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: df['temp*days'].sum() / df['days_in_month'].sum())
/tmp/ipykernel_1087/3573682381.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: df['temp_max*days'].sum() / df['days_in_month'].sum())
/tmp/ipykernel_1087/3573682381.py:32: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. 

,date,precipitation (mm),sunshine (min),mean temperature (°C),mean max daily temperature (°C),mean min daily temperature (°C)
0,1915-05-01,547.7,54361.0,9.0,14.6,3.6
1,1916-05-01,595.3,51709.0,7.9,13.1,3.0
2,1917-05-01,505.0,63540.0,10.1,15.7,4.5
3,1918-05-01,457.1,54944.0,8.6,14.0,3.5
4,1919-05-01,448.2,61665.0,8.4,14.4,3.0
...,...,...,...,...,...,...
106,2021-05-01,533.8,50780.0,10.5,16.1,5.7
107,2022-05-01,491.4,56842.0,12.1,17.9,6.9
108,2023-05-01,595.9,52639.0,12.2,18.0,7.1
109,2024-05-01,579.0,47106.0,11.6,17.1,6.9


In [21]:
weather_sion_summer.to_csv(os.path.join(folder_path, "weather_data_sion_summer.csv"), index = False)
weather_sion_winter.to_csv(os.path.join(folder_path, "weather_data_sion_winter.csv"), index = False)

weather_davos_summer.to_csv(os.path.join(folder_path, "weather_data_davos_summer.csv"), index = False)
weather_davos_winter.to_csv(os.path.join(folder_path, "weather_data_davos_winter.csv"), index = False)
